# Lecture 8 Inheritance

**Learning Objectives:**
* Implement single, multi-level, and multiple inheritance
* Apply the `super()` function to extend the behavior of the parent class
* Understand method overriding, polymorphism, and method resolution order

* Basic Syntax: 
```python
# Parent / Base / Super Class
class Parent1: 
    pass
    
class Parent2: 
    pass

# Child / Derived / Sub Class
class Child(Parent1, Parent2, ...):
    pass
```

## Polymorphism 
* objects of different classes can be treated as objects of a common superclass
* different classes can define the same method but implement the method differently

In [1]:
class Animal: 
    def __init__(self, name, age=None):
        self.name = name
        self.age = age
        
    def __str__(self): 
        return f"{type(self).__name__} (name: {self.name}, age: {self.age})"
        
    def __repr__(self): 
        return f"{type(self).__name__}(name='{self.name}', age={self.age})"

    # abstract-alike method
    def speak(self): 
        raise NotImplementedError("Will be implemented in subclasses.")

class Cat(Animal): 
    def speak(self): 
        print(f"{self.name}: meow!")
        
class Dog(Animal): 
    def __init__(self, name, color, age=None): 
        # super() returns a proxy object that routes method calls to the appropriate* parent class
        # print(super()) 
        super().__init__(name, age) # Animal.__init__(self, name, age)
        self.color = color

    # override __repr__ from Animal
    def __repr__(self): 
        return f"{type(self).__name__}(name='{self.name}', age={self.age}, color='{self.color}')"

    # override speak from Animal
    def speak(self): 
        print(f"{self.name}: bark!")

In [2]:
# Inherit everything (attributes & methods) from its parent class
# inherit Animal __init__ 
luna = Cat("luna", 1)
# inherit Animal __repr__
luna

Cat(name='luna', age=1)

In [3]:
# inherit Animal __str__
print(luna)

Cat (name: luna, age: 1)


In [4]:
luna.speak()

luna: meow!


In [5]:
fido = Dog("fido", "white", 10)
fido

Dog(name='fido', age=10, color='white')

In [6]:
fido.speak()

fido: bark!


In [7]:
animals = [luna, fido]
for animal in animals: 
    animal.speak()

luna: meow!
fido: bark!


### `isinstance()` vs. `type()`

In [8]:
# return True if object is an instance of a class or any of its subclasses
# does consider inheritance 
isinstance(fido, Animal)

True

In [9]:
# type() returns the exact type of the object
# does NOT consider inheritance 
type(fido) == Animal

False

## Multi-level Inheritance

In [10]:
class Vehicle:
    def __init__(self, brand):
        self.brand = brand

    def drive(self):
        return f"{self.brand} is moving." # A

class Car(Vehicle):
    def __init__(self, brand, fuel_type):
        super().__init__(brand) 
        self.fuel_type = fuel_type

    def drive(self):
        parent_message = super().drive() 
        return f"{parent_message} The {self.brand} runs on {self.fuel_type}." # B

class ElectricCar(Car):
    def __init__(self, brand, battery_capacity):
        super().__init__(brand, fuel_type="Electric") 
        self.battery_capacity = battery_capacity

    def drive(self):
        parent_message = super().drive()
        return f"{parent_message} It has a {self.battery_capacity} kWh battery." # C

In [11]:
electric_car = ElectricCar("Tesla", 100)
electric_car.drive()

'Tesla is moving. The Tesla runs on Electric. It has a 100 kWh battery.'

In [12]:
# What if ElectricCar doesn't have the drive method? 
electric_car = ElectricCar("Tesla", 100)
electric_car.drive()

'Tesla is moving. The Tesla runs on Electric. It has a 100 kWh battery.'

In [13]:
# What if Car doesn't have the drive method? 
electric_car = ElectricCar("Tesla", 100)
electric_car.drive()

'Tesla is moving. The Tesla runs on Electric. It has a 100 kWh battery.'

In [14]:
# What if Vehicle doesn't have the drive method? 
electric_car = ElectricCar("Tesla", 100)
electric_car.drive()

'Tesla is moving. The Tesla runs on Electric. It has a 100 kWh battery.'

In [15]:
# What if Vehicle and Car both don't have the drive method? 
electric_car = ElectricCar("Tesla", 100)
electric_car.drive()

'Tesla is moving. The Tesla runs on Electric. It has a 100 kWh battery.'

### What determines the search order when a method is invoked?
* Method resolution order: `<class>.mro()` or `<class>.__mro__`
* Invoke the first appearance of the method along the MRO
* super() also follows the MRO when routes method calls to parent class

In [16]:
ElectricCar.mro()

[__main__.ElectricCar, __main__.Car, __main__.Vehicle, object]

## Multiple Inheritance
* It's possible to have multiple parents in Python
* **NOT** possible in Java: you can only have one parent, but you can have multiple interface

In [17]:
class Animal: 
    def __init__(self, name, age=None):
        self.name = name
        self.age = age
        
    def __str__(self): 
        return f"{type(self).__name__} (name: {self.name}, age: {self.age})"
        
    def __repr__(self): 
        return f"{type(self).__name__}(name='{self.name}', age={self.age})"

    def speak(self): 
        raise NotImplementedError("Will be implemented in subclasses.")

class Pet: 
    def __init__(self, owner):
        self.owner = owner

    def speak(self):
        return f"This pet belongs to {self.owner}."

class Dog(Animal, Pet): 
    def __init__(self, name, color, owner, age=None): 
        super().__init__(name, age)
        Pet.__init__(self, owner)
        self.color = color

    def __repr__(self): 
        return f"{type(self).__name__}(name='{self.name}', age={self.age}, color='{self.color}')"

    def speak(self): 
        print(f"{self.name}: bark! {Pet.speak(self)}")

In [18]:
fido = Dog("fido", "white", "Alice", 10)
fido

Dog(name='fido', age=10, color='white')

In [19]:
fido.speak()

fido: bark! This pet belongs to Alice.


In [20]:
# Again, super() follows MRO
Dog.mro()

[__main__.Dog, __main__.Animal, __main__.Pet, object]

### The Diamond Problem
<img src="diamond_problem.png" width="150">

If B, C, D both have the same method, which one is D overriding, B or C? 
* MRO will tell you!

MRO is determined using C3 Linearization algorithm (you don't need to know)
* Generally, left to right, bottom to top

In [21]:
class A:
    def say_hello(self):
        print("Hello from A")

class B(A):
    def say_hello(self):
        print("Hello from B")
        super().say_hello()

class C(A):
    def say_hello(self):
        print("Hello from C")
        super().say_hello()

class D(B, C):
    def say_hello(self):
        print("Hello from D")
        super().say_hello()

d = D()
d.say_hello() 

Hello from D
Hello from B
Hello from C
Hello from A


In [22]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]